# Découvrir Jupiter

![Jupiter](https://upload.wikimedia.org/wikipedia/commons/e/e2/Jupiter.jpg)

![doh](https://media.giphy.com/media/xT5LMESsx1kUe8Hiyk/giphy.gif)

## Oh Mordel de Berde ! Je reprends ! 

# Découvrir Jupyter

Bon, maintenant qu'on a fait la p'tite blagounette bien naze, je pense qu'on peut faire un tour rapide des possibilités avec Jupyter. 

C'est un bon moyen de tester du code en direct (avec des sorties graphiques en utilisant *Matplotlib* ou *Bokeh*, comme on le verra par la suite) et surtout un support assez souple pour faire des cours ou des TD ou encore illustrer, avec des sorties dynamiques, une démarche techniques ou scientifiques. 

On peut aussi l'utiliser pour produire des sorties d'un modèle en changeant de manière dynamique les paramètres d'entrée. Donc on peut rendre un modèle accessible sans pour autant devoir développer une application pour cela (*à tout le moins, un notebook Jupyter pourrait servir de bêta*). 

Il peut aussi permettre de sortir une page HTML sans trop d'effort, et donc donner une sortie avec explications et illustrations dynamiques qui pourra être vue par n'importe qui disposant d'un navigateur un tant soit peu correct. 

Donc Jupyter = Markdown, Python, HTML, Latex, ... Bref, du lourd mec ! 

![noice](https://media.giphy.com/media/PhKhSXofSAm3e/giphy-tumblr.gif) 

> *Alors petit aparté, mais pour ceux qui ne connaissent pas la référence précédente, s'agirait de se mettre au parfum ou d'aller se pendre, enfin je dis ça je dis rien, hein, mais bon ... *

## Intégrer une vidéo ? Allez ! 

Alors ils ont tout prévu, donc on va utiliser les outils à disposition ! 

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('dSjvK-Z3o3U')

## Exécuter du code et faire une sortie sympa, un jeu d'enfant !

Alors bon je dis un jeu d'enfant, peut-être pas tant que ça, mais faut bien vous mentir un peu pour vous motiver hein ! 
Et puis, vous êtes des spécialistes de l'informatique, donc aucun problème pour vous ! 

Bref, ceci étant dit, on va faire un peu de géographie histoire que j'en place une et que je tente de faire le malin (*oui c'est exact, "tenter" sous-entend un échec potentiel, c'est normal, parce que je fais le malin mais j'ai conscience d'être mauvais quand même*)

Bref, on affiche des bâtiments issus d'OpenStreetMap sur un fond carto et tout ça en dynamique. 

On va utiliser [osmnx](http://osmnx.readthedocs.io/en/stable/osmnx.html) et [bokeh](https://bokeh.pydata.org/en/latest/index.html) et on va s'appuyer sur ce [tuto](https://automating-gis-processes.github.io/2016/Lesson5-interactive-map-bokeh.html) pas piqué des vers (*ouais j'aime bien "pas piqué des hannetons" mais c'est copyrighté Amixem désormais*) ! 

In [ ]:
import osmnx as ox

#On chope les bâtiments de l'INSA
adresse = "8 Avenue Jean Capelle O, Villeurbanne, France"
distance = 100 #metres
batiment = ox.buildings.buildings_from_address(adresse, distance, retain_invalid=False)

Bon ben voyons ce qu'il y a là-dedans !

In [ ]:
batiment

Et bien des données OSM sous la forme d'un GeoDataframe (Dataframe [pandas](https://pandas.pydata.org) avec une géometrie). D'ailleurs, en pensant, si vous ne connaissez pas Pandas, je vous conseille d'aller jeter un oeil, car cette librairie permet des traitements très rapides sous Python grâce à l'usage de tableaux (*array*).  

Bon, comment on affiche ça de manière sympathique ? Ben avec Bokeh tiens, mais vous ne suivez rien nondidju !!!

Il va falloir faire quelques ajustements pour que tout cela colle avec la manière de fonctionner de Bokeh (*tout se gère par l'objet ColumnDataSource*) et notamment reprojeter en EPSG: 3857. Pourquoi la projection c'est important ? Ben pour [ça](https://bl.ocks.org/mbostock/3711652). 

Une mauvaise projection peut fausser tous les calculs de distance par exemple (*notamment si vous êtes dans une projection qui fonctionne avec des degrés et que vous voulez utiliser des mètres*).

Bon notez qu'on peut utiliser directement Matplotlib quand même pour un premier aperçu ! 

> Alors petit avertissement pour les foufous qui voudraient tester Bokeh dans toute sa splendeur, il faut savoir que certains widgets qui lancent des fonctions d'update ne fonctionneront pas dans un notebook, il faudra lancer une appplication via Bokeh serve (*ceux que ça intéresse, on pourra en parler ou ils pourront aller se renseigner directement sur le site de Bokeh*)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

batiment.plot()

Bref revenons à nos moutons et à leur projection. On vérifie d'abord que le GeoDataframe dispose d'une projection, et ensuite on va le reprojeter en 3857. 

In [ ]:
batiment.crs

In [ ]:
batiment = batiment.to_crs({'init': 'epsg:3857'})
batiment

Et hop, on peut déjà constater que les coordonnées n'ont plus la même tronche. Et un petit plot() le montre même ! Bon on essaie ça et après c'est parti pour du dynamique avec Bokeh. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

batiment.plot()

Bon maintenant on utilise une fonction bien pratique pour transformer tout ça au format souhaité par Bokeh et ensuite on construit la figure avec un fond de plan OSM/[Stamen](http://maps.stamen.com/#terrain/12/37.7706/-122.3782) qui est déjà intégré dans les paramètres utilisables sous Bokeh. 

In [ ]:
#On pique discrètement une petite fonction ici:
#(https://automating-gis-processes.github.io/2016/Lesson5-interactive-map-bokeh.html)
from bokeh.models import ColumnDataSource

def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

#Et on transforme notre jeu de données
batiment['x'] = batiment.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
batiment['y'] = batiment.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

bat_df = batiment.drop('geometry', axis=1).copy()
bat_source = ColumnDataSource(bat_df)

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.tile_providers import STAMEN_TERRAIN_RETINA

output_notebook()

p = figure(title="Test",plot_height=800, plot_width=800)

p.patches('x', 'y', source=bat_source,
         fill_color="green",
         fill_alpha=1.0, line_color="black", line_width=0.05)

p.add_tile(STAMEN_TERRAIN_RETINA, alpha=0.5)

show(p)

## De la 3D vous dites ? 

Alors vite fait pour le fun, on va utiliser [Cesium.js](https://cesiumjs.org) mais dans Python (*via la librairie [cesiumpy](https://github.com/sinhrks/cesiumpy)). 

Du coup, faut reprojeter en 4326 pour coller avec les paramètres de Cesium (heureusement c'est simple), la petite fonction qui va bien et hop !

> **Alors petit avertissement avant de lancer: ça risque de vous bouffer pas mal de ram, donc faites attention.**

> Si vous avez une erreur JavaScript, pas de panique, refaites l'exécution du code. 

In [ ]:
import itertools
import cesiumpy

batiment = batiment.to_crs({"init":"epsg:4326"})

def poly_to_cesium(poly):
    coords = poly.exterior.coords.xy
    l_coords = zip(list(coords[0]),list(coords[1]))
    l_coords = list(itertools.chain(*l_coords))
    p = cesiumpy.Polygon(hierarchy = l_coords, extrudedHeight = 10)
    
    return p
    

batiment["cesium_poly"] = batiment["geometry"].apply(poly_to_cesium)

v = cesiumpy.Viewer()
# Transform gdf Serie to list
l_poly = batiment.cesium_poly.tolist()

for poly in l_poly:
    v.entities.add(poly)
    
v